In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/m4-forecasting-competition-dataset/Quarterly-test.csv
/kaggle/input/m4-forecasting-competition-dataset/m4_info.csv
/kaggle/input/m4-forecasting-competition-dataset/Weekly-train.csv
/kaggle/input/m4-forecasting-competition-dataset/Hourly-train.csv
/kaggle/input/m4-forecasting-competition-dataset/Daily-train.csv
/kaggle/input/m4-forecasting-competition-dataset/Yearly-test.csv
/kaggle/input/m4-forecasting-competition-dataset/Yearly-train.csv
/kaggle/input/m4-forecasting-competition-dataset/Monthly-train.csv
/kaggle/input/m4-forecasting-competition-dataset/Weekly-test.csv
/kaggle/input/m4-forecasting-competition-dataset/Daily-test.csv
/kaggle/input/m4-forecasting-competition-dataset/Hourly-test.csv
/kaggle/input/m4-forecasting-competition-dataset/Quarterly-train.csv
/kaggle/input/m4-forecasting-competition-dataset/Monthly-test.csv


In [2]:
!pip install pmdarima prophet tensorflow==2.*

import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
import pmdarima as pm
from prophet import Prophet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 34.6 MB/s eta 0:00:00


In [3]:
# load datat
info_df  = pd.read_csv('/kaggle/input/m4-forecasting-competition-dataset/m4_info.csv')\
               .set_index('M4id')
train_df = pd.read_csv('/kaggle/input/m4-forecasting-competition-dataset/Quarterly-train.csv')
test_df  = pd.read_csv('/kaggle/input/m4-forecasting-competition-dataset/Quarterly-test.csv')
print(f"Loaded {len(train_df)} Quarterly series")


Loaded 24000 Quarterly series


In [4]:

def row_to_series(df_row):
    """Convert one CSV row to a 1D float numpy array (drops NaNs)."""
    return df_row.iloc[1:].dropna().astype(float).values

def compute_errors(y_true, y_pred):
    """Return MAE, RMSE, MAPE between two equal-length arrays."""
    mae  = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mask = y_true != 0
    mape = np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100
    return mae, rmse, mape


In [5]:
# arima and ets evaluation 
import sys
import os
import warnings
import logging


warnings.filterwarnings("ignore")
os.environ['PYTHONWARNINGS'] = 'ignore'
logging.basicConfig(level=logging.CRITICAL)


import statsmodels.api as sm
sm.tools.sm_exceptions.ConvergenceWarning = lambda *a, **k: None

from contextlib import contextmanager
from joblib import Parallel, delayed
from tqdm.auto import tqdm
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import multiprocessing as mp

#  manager for  warnings
@contextmanager
def suppress_all_output():
    """Suppress all output including warnings"""
    with open(os.devnull, 'w') as devnull:
        old_stdout = sys.stdout
        old_stderr = sys.stderr
        try:
            sys.stdout = devnull
            sys.stderr = devnull
            yield
        finally:
            sys.stdout = old_stdout
            sys.stderr = old_stderr

# data
info_df  = pd.read_csv('/kaggle/input/m4-forecasting-competition-dataset/m4_info.csv').set_index('M4id')
train_df = pd.read_csv('/kaggle/input/m4-forecasting-competition-dataset/Quarterly-train.csv')
test_df  = pd.read_csv('/kaggle/input/m4-forecasting-competition-dataset/Quarterly-test.csv')


def row_to_series(df_row):
    return df_row.iloc[1:].dropna().astype(float).values

def compute_errors(y_true, y_pred):
    mae  = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mask = y_true != 0
    if np.any(mask):
        mape = np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100
    else:
        mape = 0.0
    return mae, rmse, mape

# arima
def silent_arima(y_train, h):
    if len(y_train) < 4:
        return np.full(h, y_train[-1])
    
    with suppress_all_output():
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            try:
                model = ARIMA(y_train, order=(1,1,1))
                fitted = model.fit(disp=0, method='innovations_mle')
                return fitted.forecast(steps=h)
            except:
                try:
                    model = ARIMA(y_train, order=(1,0,0))
                    fitted = model.fit(disp=0)
                    return fitted.forecast(steps=h)
                except:
                    return np.full(h, y_train[-1])

# ets
def silent_ets(y_train, h):
    if len(y_train) < 4:
        return np.full(h, y_train[-1])
    
    with suppress_all_output():
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            
            # pre check 
            if np.std(y_train) < 1e-10:
                return np.full(h, y_train[-1])
            
            try:
                # exponential smoothing
                model = ExponentialSmoothing(
                    y_train,
                    trend=None,
                    seasonal=None,
                    initialization_method='heuristic'
                )
                fitted = model.fit(optimized=False, smoothing_level=0.3)
                return fitted.forecast(steps=h)
            except:
                # Moving average fallback
                window = min(4, len(y_train))
                return np.full(h, np.mean(y_train[-window:]))

#  single series
def process_series_silent(idx):
    try:
        train_row = train_df.iloc[idx]
        test_row  = test_df.iloc[idx]
        y_train   = row_to_series(train_row)
        y_test    = row_to_series(test_row)
        h         = len(y_test)
        
        if h == 0:
            return np.nan, np.nan, np.nan, np.nan, np.nan, np.nan
        
        # arima
        f_ar = silent_arima(y_train, h)
        ar_mae, ar_rmse, ar_mape = compute_errors(y_test, f_ar)
        
        # ets
        f_ets = silent_ets(y_train, h)
        ets_mae, ets_rmse, ets_mape = compute_errors(y_test, f_ets)
        
        return ar_mae, ar_rmse, ar_mape, ets_mae, ets_rmse, ets_mape
        
    except:
        return np.nan, np.nan, np.nan, np.nan, np.nan, np.nan

# parallel process
n_series = len(train_df)
n_jobs = min(4, mp.cpu_count())
print(f"Processing {n_series} series with {n_jobs} workers...")
print("Using ARIMA(1,1,1) and Simple Exponential Smoothing")


results = Parallel(n_jobs=n_jobs, backend='loky', batch_size='auto')(
    delayed(process_series_silent)(i) for i in tqdm(range(n_series), desc="Forecasting")
)

# results
ar_mae, ar_rmse, ar_mape, ets_mae, ets_rmse, ets_mape = zip(*results)

# filter results
ar_valid = [(mae, rmse, mape) for mae, rmse, mape in zip(ar_mae, ar_rmse, ar_mape) 
            if not np.isnan(mae)]
ets_valid = [(mae, rmse, mape) for mae, rmse, mape in zip(ets_mae, ets_rmse, ets_mape) 
             if not np.isnan(mae)]

# averages and display results
print("\n" + "="*50)
print("FORECASTING RESULTS")
print("="*50)

if ar_valid:
    ar_avg = np.mean(ar_valid, axis=0)
    print(f"\nARIMA(1,1,1) Performance:")
    print(f"  - Average MAE:  {ar_avg[0]:,.2f}")
    print(f"  - Average RMSE: {ar_avg[1]:,.2f}")
    print(f"  - Average MAPE: {ar_avg[2]:.2f}%")

if ets_valid:
    ets_avg = np.mean(ets_valid, axis=0)
    print(f"\nExponential Smoothing Performance:")
    print(f"  - Average MAE:  {ets_avg[0]:,.2f}")
    print(f"  - Average RMSE: {ets_avg[1]:,.2f}")
    print(f"  - Average MAPE: {ets_avg[2]:.2f}%")

print(f"\nValid results: ARIMA {len(ar_valid)}/{n_series}, ETS {len(ets_valid)}/{n_series}")

#  relative mae (mae/mean)
print("\n" + "="*50)
print("RELATIVE PERFORMANCE METRICS")
print("="*50)

# average of series means
series_means = []
for i in range(len(train_df)):
    series = row_to_series(train_df.iloc[i])
    if len(series) > 0:
        series_means.append(np.mean(series))

avg_series_mean = np.mean(series_means)
print(f"\nDataset Statistics:")
print(f"  - Average series mean: {avg_series_mean:,.2f}")
print(f"  - Number of series: {n_series:,}")

if ar_valid:
    ar_mae_values = [x[0] for x in ar_valid]
    ar_mae_mean = np.mean(ar_mae_values)
    relative_mae_arima = ar_mae_mean / avg_series_mean
    print(f"\nARIMA Relative Performance:")
    print(f"  - Average MAE: {ar_mae_mean:,.2f}")
    print(f"  - Relative MAE (MAE/mean): {relative_mae_arima:.4f} ({relative_mae_arima*100:.2f}%)")

if ets_valid:
    ets_mae_values = [x[0] for x in ets_valid]
    ets_mae_mean = np.mean(ets_mae_values)
    relative_mae_ets = ets_mae_mean / avg_series_mean
    print(f"\nETS Relative Performance:")
    print(f"  - Average MAE: {ets_mae_mean:,.2f}")
    print(f"  - Relative MAE (MAE/mean): {relative_mae_ets:.4f} ({relative_mae_ets*100:.2f}%)")

pr_mae, pr_rmse, pr_mape = ets_mae, ets_rmse, ets_mape

print("\n" + "="*50)

import gc
import time
gc.collect()
time.sleep(1)
gc.collect()
print("\nAll done!")

Processing 24000 series with 4 workers...
Using ARIMA(1,1,1) and Simple Exponential Smoothing


Forecasting:   0%|          | 0/24000 [00:00<?, ?it/s]


FORECASTING RESULTS

ARIMA(1,1,1) Performance:
  - Average MAE:  655.56
  - Average RMSE: 767.78
  - Average MAPE: 13.18%

Exponential Smoothing Performance:
  - Average MAE:  712.89
  - Average RMSE: 815.67
  - Average MAPE: 14.49%

Valid results: ARIMA 24000/24000, ETS 24000/24000

RELATIVE PERFORMANCE METRICS

Dataset Statistics:
  - Average series mean: 4,141.00
  - Number of series: 24,000

ARIMA Relative Performance:
  - Average MAE: 655.56
  - Relative MAE (MAE/mean): 0.1583 (15.83%)

ETS Relative Performance:
  - Average MAE: 712.89
  - Relative MAE (MAE/mean): 0.1722 (17.22%)


All done!
